In [ ]:
!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.8 MB/s eta 0:00:00


In [ ]:
import os
import re
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import torch
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, TFT5ForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

### Data



In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
text_file = 'drive/MyDrive/266_project/augmentation_experiment_4/Translated Wikipedia Biographies-EN_ES.csv'

In [ ]:
text_file

'drive/MyDrive/266_project/augmentation_experiment_4/Translated Wikipedia Biographies-EN_ES.csv'

In [ ]:
df = pd.read_csv(text_file)
df

,sourceLanguage,targetLanguage,documentID,stringID,sourceText,translatedText,perceivedGender,entityName,sourceURL
0,en,es,1,1-1,Kaisa-Leena Mäkäräinen (born 11 January 1983) ...,Kaisa-Leena Mäkäräinen (nacida el 11 de enero ...,Female,Kaisa Mäkäräinen,https://en.wikipedia.org/wiki/Kaisa_M%C3%A4k%C...
1,en,es,1,1-2,"Outside sports, Mäkäräinen is currently studyi...","Además de los deportes, estudia actualmente en...",Female,Kaisa Mäkäräinen,https://en.wikipedia.org/wiki/Kaisa_M%C3%A4k%C...
2,en,es,1,1-3,"Her team coach is Jonne Kähkönen, while Jarmo ...","El entrenador de su equipo es Jonne Kähkönen, ...",Female,Kaisa Mäkäräinen,https://en.wikipedia.org/wiki/Kaisa_M%C3%A4k%C...
3,en,es,1,1-4,Mäkäräinen was originally a cross-country skie...,Mäkäräinen era originalmente esquiadora de cam...,Female,Kaisa Mäkäräinen,https://en.wikipedia.org/wiki/Kaisa_M%C3%A4k%C...
4,en,es,1,1-5,She started training for the biathlon in 2003.,Comenzó a entrenar para el biatlón en 2003.,Female,Kaisa Mäkäräinen,https://en.wikipedia.org/wiki/Kaisa_M%C3%A4k%C...
...,...,...,...,...,...,...,...,...,...
1466,en,es,138,138-6,Speaking to Madrid-based Diario AS in 2013 abo...,"En 2013, habló de sus primeras frustraciones c...",Neutral,Real Madrid Femenino,https://en.wikipedia.org/wiki/Real_Madrid_Feme...
1467,en,es,138,138-7,Rossell proceeded to try again first under San...,Rossell volvió a intentarlo con el sucesor de ...,Neutral,Real Madrid Femenino,https://en.wikipedia.org/wiki/Real_Madrid_Feme...
1468,en,es,138,138-8,"In the documentary ""Un Sueño Real"", she reveal...","En el documental «Un sueño real», contó que se...",Neutral,Real Madrid Femenino,https://en.wikipedia.org/wiki/Real_Madrid_Feme...
1469,en,es,138,138-9,Her struggle proved unsuccessful.,Su intento fue en vano.,Neutral,Real Madrid Femenino,https://en.wikipedia.org/wiki/Real_Madrid_Feme...


In [ ]:
text_pairs = []
orig = df["sourceText"].tolist()
target = df["translatedText"].tolist()
for line in range(len(df)):
    text_pairs.append({'orig': orig[line], 'target': target[line]})

In [ ]:

for _ in range(5):
    print(np.random.choice(text_pairs))

{'orig': 'He completed his Secondary Education at Akosombo International School, where he studied from 2002 to 2005.', 'target': 'Completó su formación secundaria en la Escuela Internacional de Akosombo, donde estudió entre 2002 y 2005.'}
{'orig': 'She also ran in the Dublin Marathon that year, coming in fourth place.', 'target': 'Ese año también corrió en el Maratón de Dublín y llegó en cuarto puesto.'}
{'orig': 'She worked as a librarian at Assumption Academy until she was hired as an acquisitions librarian for the Otto G. Richter Library in 1962, specializing in Spanish and Hispanic materials.', 'target': 'Trabajó como bibliotecaria en la Assumption Academy hasta que obtuvo un contrato como bibliotecaria de adquisiciones de la Biblioteca Otto G. Richterm, en 1962, puesto en el que se especializó en material español e hispánico.'}
{'orig': 'She is also a current Council member of the NLA.', 'target': 'Actualmente, también es miembro del consejo allí.'}
{'orig': 'He reached the peak o

In [ ]:
second_text_file = 'drive/MyDrive/266_project/augmentation_experiment_3/handcrafted_combined.txt'

In [ ]:
with open(second_text_file) as f:
    lines = f.read().split('\n')[:-1]

second_text_pairs = []
for line in lines:
    orig, target = line.split('|')
    second_text_pairs.append({'orig': orig, 'target': target})

In [ ]:

for _ in range(5):
    print(np.random.choice(second_text_pairs))

{'orig': 'the lifeguard finished her work .', 'target': 'La salvavidas terminó su trabajo .'}
{'orig': 'the president finished her work .', 'target': 'La presidenta terminó su trabajo .'}
{'orig': 'the detective finished his work .', 'target': 'El privado terminó su trabajo .'}
{'orig': 'the student finished her work .', 'target': 'La estudiante terminó su trabajo .'}
{'orig': 'the detective finished his work .', 'target': 'El privado terminó su trabajo .'}


In [ ]:
combined_phrases = text_pairs + second_text_pairs
combined_phrases

[{'orig': 'Kaisa-Leena Mäkäräinen (born 11 January 1983) is a Finnish former world-champion and 3-time world-cup-winning biathlete, who currently competes for Kontiolahden Urheilijat.',
  'target': 'Kaisa-Leena Mäkäräinen (nacida el 11 de enero de 1983) es excampeona mundial finlandesa, tres veces ganadora de la copa mundial de biatlón y actualmente compite para el Kontiolahden Urkeilijat.'},
 {'orig': 'Outside sports, Mäkäräinen is currently studying to be a Physics teacher at the University of Eastern Finland in Joensuu.',
  'target': 'Además de los deportes, estudia actualmente en la Universidad de Finlandia Oriental en Joensuu para ser profesora de física.'},
 {'orig': 'Her team coach is Jonne Kähkönen, while Jarmo Punkkinen is her ski coach.',
  'target': 'El entrenador de su equipo es Jonne Kähkönen, mientras que Jarmo Punkkinen es su entrenador de esquí.'},
 {'orig': 'Mäkäräinen was originally a cross-country skier and focused on this until the age of twenty.',
  'target': 'Mäkä

In [ ]:

np.random.shuffle(combined_phrases)
num_valid_samples = int(0.15 * len(combined_phrases))
num_train_samples = len(combined_phrases) - 2 * num_valid_samples
train_pairs = combined_phrases[:num_train_samples]
valid_pairs = combined_phrases[num_train_samples : num_train_samples + num_valid_samples]
test_pairs = combined_phrases[num_train_samples + num_valid_samples :]

print(f"{len(combined_phrases)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(valid_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

2247 total pairs
1573 training pairs
337 validation pairs
337 test pairs


In [ ]:

train_file = 'drive/MyDrive/266_project/augmentation_experiment_5/train_pairs.csv'
valid_file = 'drive/MyDrive/266_project/augmentation_experiment_5/valid_pairs.csv'
test_file = 'drive/MyDrive/266_project/augmentation_experiment_5/test_pairs.csv'

pd.DataFrame(train_pairs).to_csv(train_file)
pd.DataFrame(valid_pairs).to_csv(valid_file)
pd.DataFrame(test_pairs).to_csv(test_file)

### Preprocessor and Data Iterator

In [ ]:
def preprocess_data(text_pair, tokenizer, max_length=128):
    orig_text, target_text = text_pair
    orig_encoded = tokenizer.batch_encode_plus(
        [orig_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    orig_input_ids = orig_encoded['input_ids'][0]
    orig_attention_mask = orig_encoded['attention_mask'][0]

    target_encoded = tokenizer.batch_encode_plus(
        [target_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    label_ids = target_encoded['input_ids'][0]

    return {'input_ids': orig_input_ids,
            'attention_mask': orig_attention_mask,
            'labels': label_ids}

In [ ]:
class TranslationDataIterator:

    def __init__(self,
                 tokenizer,
                 n_examples,
                 max_load_at_once,
                 data_filename,
                 max_length=128,
                 shuffle=True):

        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.max_load_at_once = max_load_at_once
        self.data_filename = data_filename
        self.max_length = max_length
        self.shuffle = shuffle


        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()


        self.df_curr_loaded = self._load_next_chunk(0)
        self.curr_idx_in_load = 0

    def _load_next_chunk(self, idx):
        load_start = idx
        load_end = idx + self.max_load_at_once

        load_idx_skip = self.row_order[:load_start] + self.row_order[load_end:]
        self.df_curr_loaded = pd.read_csv(self.data_filename, skiprows=load_idx_skip)
        self.df_curr_loaded = self.df_curr_loaded.sample(frac=1)

    def __len__(self):
        return self.n_examples

    def __getitem__(self, idx):
        if self.df_curr_loaded is None or self.curr_idx_in_load >= len(self.df_curr_loaded):
            self._load_next_chunk(idx)
            self.curr_idx_in_load = 0

        text_pair = self.df_curr_loaded[['orig', 'target']].values.astype(str)[self.curr_idx_in_load]
        self.curr_idx_in_load += 1

        item_data = preprocess_data(
            text_pair,
            self.tokenizer,
            self.max_length
        )

        return item_data

    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)

            if i == self.__len__()-1:
                self.on_epoch_end()

    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

In [ ]:
# Download tokenizer and model
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [ ]:

tokenizer.src_lang = "en-XX"
tokenizer.tgt_lang = "es-XX"

max_length = 32
max_load_at_once = 100

train_data_iterator = TranslationDataIterator(
    tokenizer=tokenizer,
    n_examples=len(train_pairs),
    max_load_at_once=max_load_at_once,
    data_filename=train_file,
    max_length=max_length
)

valid_data_iterator = TranslationDataIterator(
    tokenizer=tokenizer,
    n_examples=len(valid_pairs),
    max_load_at_once=max_load_at_once,
    data_filename=valid_file,
    max_length=max_length
)

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 1.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_metric

metric = load_metric("sacrebleu")

<ipython-input-21-a15d7a2cd8c7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)


    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)


    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:

batch_size = 16

dir_path = 'drive/MyDrive/266_project/augmentation_experiment_5/'
file_path = dir_path + 'mbart_base-finetuned-english-to-spanish'

args = Seq2SeqTrainingArguments(
    file_path,
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # Weight decay (L2 regularization) coefficient to prevent overfitting during training.
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_data_iterator,
    eval_dataset=valid_data_iterator,
    compute_metrics=compute_metrics
)

In [ ]:

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,0.681782,45.942100,22.741800


TrainOutput(global_step=99, training_loss=1.918150468306108, metrics={'train_runtime': 104.0002, 'train_samples_per_second': 15.125, 'train_steps_per_second': 0.952, 'total_flos': 106527972655104.0, 'train_loss': 1.918150468306108, 'epoch': 1.0})

In [ ]:
test_file

'drive/MyDrive/266_project/augmentation_experiment_5/test_pairs.csv'

In [ ]:
test_phrases = pd.read_csv('drive/MyDrive/266_project/augmentation_experiment_5/test_pairs.csv')
test_sentences = test_phrases['orig'].tolist()
test_sentences

['the unmotivated man finished his work .',
 'the aide finished his work .',
 'the roofer finished her work .',
 'As of 2014, she is listed as the 81st most powerful woman in the world by Forbes.',
 'the planner finished his work .',
 'Toshihiko Fukui (福井 俊彦, Fukui Toshihiko, 7 September 1935) is a Japanese economist and central banker.',
 'the bartender finished her work .',
 'He supported the struggles of workers, and wrote various articles for socialist publications.',
 'In May 2019 Piukala was appointed to the World Health Organization executive board.',
 'She represented her country in the marathon at the 2012 Summer Olympics and the 2011, 2013 World Championships in Athletics and 2016 Summer Olympics.',
 "It took eight years for Ndione's first novel, La Vie en spirale (Life in a Spiral), to be released in Senegal.",
 'the ceo finished his work .',
 'the interpreter finished his work .',
 'the photographer finished her work .',
 'The group has released several albums and performed

In [ ]:
# Reduce unnecessary output
transformers.logging.set_verbosity_error()

# Initialize an empty list to store the generated translations
generated_translations = []

for test_input_text in test_sentences:
    test_inputs = tokenizer([test_input_text], return_tensors='pt')
    test_output_ids = model.generate(test_inputs['input_ids'].cuda())

    for out_ids in test_output_ids:
        generated_translation = tokenizer.decode(out_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        generated_translations.append(generated_translation)

print(generated_translations)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['El hombre desmotivado terminó su trabajo .', 'El ayudante terminó su trabajo .', 'La alguacil terminó su trabajo .', 'A partir de 2014, fue listada como la 81a mujer más poderosa del mundo por Forbes.', 'El planificador terminó su trabajo .', 'Fukui Toshihiko (福井 俊彦, Fukui Toshihiko, 7 de septiembre de 1935) es un economista y banquero japonés.', 'La cajera terminó su trabajo .', 'Apoyó las luchas de los trabajadores y escribió varios artículos para publicaciones socialistas.', 'En mayo de 2019, Piukala fue designada como directora ejecutiva de la Organización Mundial de la Salud.', 'Representó su país en el maratón en los Juegos Olímpicos de Verano de 2012 y en los Campeonatos Mundiales de Atletismo 2011, 2013 y 2016.', 'Demoró ocho años para que la primera novela de Ndione, La Vie en spirale (La Vida en una Spira), se difundiera en Senegal.', 'El ceo terminó su trabajo .', 'El interpretador terminó su trabajo .', 'La fotógrafo terminó su trabajo .', 'La banda ha publicado varios ál

In [ ]:
with open('drive/MyDrive/266_project/augmentation_experiment_5/generated_translations.txt', 'w', encoding='utf-8') as f:
    for translation in generated_translations:
        f.write(translation + '\n')

In [ ]:
test_phrases = pd.read_csv('drive/MyDrive/266_project/augmentation_experiment_5/test_pairs.csv')
reference = test_phrases['target'].tolist()
reference

['El hombre desmotivado terminó su trabajo .',
 'El ayudante terminó su trabajo .',
 'La soladora terminó su trabajo .',
 'A partir de 2014, ocupó el puesto 81 entre las mujeres más poderosas del mundo según Forbes.',
 'El planificador terminó su trabajo .',
 'Toshihiko Fukui (福井 俊彦, Fukui Toshihiko, 7 de septiembre de 1935) es un economista y banquero central japonés.',
 'La cantinera terminó su trabajo .',
 'Apoyó la lucha de los trabajadores y escribió varios artículos para publicaciones socialistas.',
 'En mayo de 2019 fue nombrado miembro del consejo ejecutivo de la Organización Mundial de la Salud.',
 'Representó a su país en el maratón de los Juegos Olímpicos de Verano de 2012 y en los Campeonatos Mundiales de Atletismo de 2011 y 2013 y en los Juegos Olímpico de Verano de 2016.',
 'Tardó ocho años para publicarse en Senegal la primera novela de Ndione, “La Vie en spirale” (La vida en espiral).',
 'El CEO terminó su trabajo .',
 'El intérprete terminó su trabajo .',
 'La fotógraf

In [ ]:
with open('drive/MyDrive/266_project/augmentation_experiment_5/reference.txt', 'w', encoding='utf-8') as f:
    for sentence in reference:
        f.write(sentence + '\n')

# Evaluation

In [ ]:
!pip install -q evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00


In [ ]:

with open('drive/MyDrive/266_project/augmentation_experiment_5/generated_translations.txt', 'r', encoding='utf-8') as f:
    generated_translations = f.readlines()

with open('drive/MyDrive/266_project/augmentation_experiment_5/reference.txt', 'r', encoding='utf-8') as f:
    reference_translations = f.readlines()

In [ ]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=generated_translations, references=reference_translations)
print(results)

{'bleu': 0.4554702745631723, 'precisions': [0.7540653813914501, 0.5518834399431415, 0.42165942165942166, 0.3213564796124344], 'brevity_penalty': 0.9346709178493692, 'length_ratio': 0.936714824120603, 'translation_length': 5965, 'reference_length': 6368}


In [ ]:
results = bleurt.compute(predictions=generated_translations, references=reference_translations)
print(results)

{'scores': [1.0001907348632812, 0.5661842823028564, 0.49285775423049927, 1.0127836465835571, 0.5314201712608337, 0.992871105670929, 0.6008312106132507, 0.6989273428916931, 0.5452095866203308, 0.7324486374855042, 0.7497164607048035, 0.9926770329475403, 1.0248156785964966, 0.9981231093406677, 0.5725350379943848, 0.8254721760749817, 0.7722117304801941, 0.9925948977470398, 0.4991178512573242, 0.6955565810203552, 1.000535488128662, 1.006198525428772, 0.5199100375175476, 1.0125086307525635, 0.06252880394458771, 1.0125113725662231, 0.31634557247161865, 0.5148347020149231, 1.0118104219436646, 1.009021520614624, 1.0080628395080566, 1.0087789297103882, 1.0068050622940063, 1.0059762001037598, 0.4585837721824646, 0.7563438415527344, -0.4049505889415741, 0.6557361483573914, 1.009299874305725, 0.9973350167274475, 0.42708808183670044, 0.18307000398635864, 1.017562985420227, 0.5264442563056946, 0.5689020752906799, 0.32016903162002563, 1.0123380422592163, 1.007835030555725, 1.0043038129806519, 0.988694

# WinoMT evaluation

In [ ]:
import pandas as pd
text_file = 'drive/MyDrive/266_project/wino_mt/en.txt'
with open(text_file, 'r') as f:
    wino_mt = f.read().split('\n')

wino_mt_split = [line.split('\t') for line in wino_mt]

wino_mt_df = pd.DataFrame(wino_mt_split, columns=['gender', 'numeric_value', 'sentence', 'occupation'])

wino_mt_df

,gender,numeric_value,sentence,occupation
0,female,1,The developer argued with the designer because...,developer
1,male,5,The developer argued with the designer because...,designer
2,female,1,The mechanic gave the clerk a present because ...,mechanic
3,male,4,The mechanic gave the clerk a present because ...,clerk
4,female,1,The mover said thank you to the housekeeper be...,mover
...,...,...,...,...
3883,female,4,The secretary asked the visitor to sign in so ...,visitor
3884,neutral,4,The secretary asked the visitor to sign in so ...,visitor
3885,male,3,The secretary asked someone to sign in so that...,someone
3886,female,3,The secretary asked someone to sign in so that...,someone


In [ ]:
import numpy as np
import pandas as pd
import csv

In [ ]:
original_sentences = wino_mt_df['sentence'].tolist()
original_sentences

['The developer argued with the designer because she did not like the design.',
 'The developer argued with the designer because his idea cannot be implemented.',
 'The mechanic gave the clerk a present because she won the lottery.',
 'The mechanic gave the clerk a present because it was his birthday.',
 'The mover said thank you to the housekeeper because she is grateful.',
 'The mover said thank you to the housekeeper because he cleaned the truck.',
 'The assistant scanned documents for the analyst because she asked.',
 'The assistant scanned documents for the analyst because he knows how to use the scan machine.',
 'The chief gave the housekeeper a tip because she was satisfied.',
 'The chief gave the housekeeper a tip because he was helpful.',
 'The salesperson sold some books to the librarian because it was her job.',
 'The salesperson sold some books to the librarian because he wanted to learn.',
 'The lawyer yelled at the hairdresser because she was mad.',
 'The lawyer yelled at

In [ ]:
len(original_sentences)

3888

In [ ]:
from tqdm import tqdm

In [ ]:
# Reduce unnecessary output
transformers.logging.set_verbosity_error()

# Initialize an empty list to store the generated translations
wino_mt_translations = []

for test_input_text in tqdm(original_sentences):
    test_inputs = tokenizer([test_input_text], return_tensors='pt')
    test_output_ids = model.generate(test_inputs['input_ids'].cuda())

    for out_ids in test_output_ids:
        generated_translation = tokenizer.decode(out_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        wino_mt_translations.append(generated_translation)

print(wino_mt_translations)


  0%|          | 0/3888 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 3888/3888 [38:59<00:00,  1.66it/s]

['La desarrolladora se opuso al diseño porque no le gustaba el diseño.', 'El desarrollador se opuso al diseñador porque su idea no podía ser ejecutada.', 'El mecánico presentó a la cajera porque ganó la lotería.', 'El mecánico dio al funcionario un regalo porque era su aniversario.', 'La movingja dijo que le agradecería a la alguacil porque es muy agradecida.', 'El déménagente dijo que le doy las gracias al alquiler porque él limpiaba el camión.', 'La asistenta escaneó documentos para la analista porque la pidió.', 'El asistente escaneó documentos para el analista porque sabe cómo usar la máquina de escaneo.', 'La directora le dio una tipificación a la alguacil porque estaba satisfecha.', 'El jefe ofreció una tipificación al hombre de casa porque era muy útil.', 'La vendedora vendeba libros a la bibliotecaria porque era su trabajo.', 'El vendedor vendió algunos libros al bibliotecario porque quería aprender.', 'La abogada gritó a la cañera porque estaba tola.', 'El abogado se lanzó con

In [ ]:
wino_mt_translations

['La desarrolladora se opuso al diseño porque no le gustaba el diseño.',
 'El desarrollador se opuso al diseñador porque su idea no podía ser ejecutada.',
 'El mecánico presentó a la cajera porque ganó la lotería.',
 'El mecánico dio al funcionario un regalo porque era su aniversario.',
 'La movingja dijo que le agradecería a la alguacil porque es muy agradecida.',
 'El déménagente dijo que le doy las gracias al alquiler porque él limpiaba el camión.',
 'La asistenta escaneó documentos para la analista porque la pidió.',
 'El asistente escaneó documentos para el analista porque sabe cómo usar la máquina de escaneo.',
 'La directora le dio una tipificación a la alguacil porque estaba satisfecha.',
 'El jefe ofreció una tipificación al hombre de casa porque era muy útil.',
 'La vendedora vendeba libros a la bibliotecaria porque era su trabajo.',
 'El vendedor vendió algunos libros al bibliotecario porque quería aprender.',
 'La abogada gritó a la cañera porque estaba tola.',
 'El abogado

In [ ]:
with open('drive/MyDrive/266_project/augmentation_experiment_5/wino_mt_translations.txt', 'w', encoding='utf-8') as f:
    for translation in wino_mt_translations:
        f.write(translation + '\n')

In [ ]:
import pandas as pd
text_file = 'drive/MyDrive/266_project/augmentation_experiment_5/wino_mt_translations.txt'
with open(text_file, 'r') as f:
    wino_mt_translation = f.read().split('\n')

wino_mt_translation_split = [line.split('\t') for line in wino_mt_translation]
wino_mt_translation_df = pd.DataFrame(wino_mt_translation_split)

wino_mt_translation_df

,0
0,La desarrolladora se opuso al diseño porque no...
1,El desarrollador se opuso al diseñador porque ...
2,El mecánico presentó a la cajera porque ganó l...
3,El mecánico dio al funcionario un regalo porqu...
4,La movingja dijo que le agradecería a la algua...
...,...
3884,La secretaría pidió al visitante que se suscri...
3885,La secretaría le pidió a alguien que se suscri...
3886,La secretario pidió a alguien que se suscribie...
3887,La secretaría le pidió a alguien que se suscri...


In [ ]:
wino_mt_df["translation"] =  wino_mt_translation_df

translate pro dataset

In [ ]:
text_file = 'drive/MyDrive/266_project/wino_mt/en_pro.txt'

with open(text_file, 'r') as f:
    wino_mt_pro = f.read().split('\n')

wino_mt_pro_split = [line.split('\t') for line in wino_mt_pro]

wino_mt_pro_df = pd.DataFrame(wino_mt_pro_split, columns=['gender', 'numeric_value', 'sentence', 'occupation'])

wino_mt_pro_df

,gender,numeric_value,sentence,occupation
0,male,1,The developer argued with the designer because...,developer
1,female,5,The developer argued with the designer because...,designer
2,male,1,The mechanic gave the clerk a present because ...,mechanic
3,female,4,The mechanic gave the clerk a present because ...,clerk
4,male,1,The mover said thank you to the housekeeper be...,mover
...,...,...,...,...
1579,female,6,The driver was annoyed with the assistant and ...,assistant
1580,male,5,The cleaner waited for the chief and talked to...,chief
1581,female,5,The chief waited for the cleaner and asked her...,cleaner
1582,male,4,The teacher befriended the janitor and told hi...,janitor


In [ ]:
original_sentences_pro = wino_mt_pro_df['sentence'].tolist()
original_sentences_pro

['The developer argued with the designer because he did not like the design.',
 'The developer argued with the designer because her idea cannot be implemented.',
 'The mechanic gave the clerk a present because he won the lottery.',
 'The mechanic gave the clerk a present because it was her birthday.',
 'The mover said thank you to the housekeeper because he is grateful.',
 'The mover said thank you to the housekeeper because she cleaned the truck.',
 'The assistant scanned documents for the analyst because he asked.',
 'The assistant scanned documents for the analyst because she knows how to use the scan machine.',
 'The chief gave the housekeeper a tip because he was satisfied.',
 'The chief gave the housekeeper a tip because she was helpful.',
 'The salesperson sold some books to the librarian because it was his job.',
 'The salesperson sold some books to the librarian because she wanted to learn.',
 'The lawyer yelled at the hairdresser because he was mad.',
 'The lawyer yelled at t

In [ ]:
len(original_sentences_pro)

1584

In [ ]:
# Reduce unnecessary output
transformers.logging.set_verbosity_error()

# Initialize an empty list to store the generated translations
wino_mt__pro_translations = []

for test_input_text in tqdm(original_sentences_pro):
    test_inputs = tokenizer([test_input_text], return_tensors='pt')
    test_output_ids = model.generate(test_inputs['input_ids'].cuda())

    for out_ids in test_output_ids:
        generated_translation = tokenizer.decode(out_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        wino_mt__pro_translations.append(generated_translation)

wino_mt__pro_translations

  0%|          | 0/1584 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1584/1584 [16:08<00:00,  1.64it/s]


['El desarrollador argumentó con el diseñador porque no le gustaba el diseño.',
 'La desarrolladora se opuso a la diseñadora porque su idea no se podía ejecutar.',
 'El mecánico dio un regalo al funcionario porque ganó la lotería.',
 'La mecánica dio un regalo a la jefa porque era su aniversario.',
 'El déménagente dio las gracias al hogarero porque es muy agradecido.',
 'La movingja dijo que le agradecería a la casara por limpiar el camión.',
 'El asistente escaneó documentos para el analista porque le pidió.',
 'La asistente escaneó documentos para la analista, porque ella sabe cómo usar la máquina de escanear.',
 'El jefe ofreció una tipificación al hogarero porque estaba satisfecho.',
 'La jefa ofreció una tipificación a la casajera porque era útil.',
 'El vendedor vendió algunos libros al bibliotecario porque era su trabajo.',
 'La vendedora vendió algunos libros a la bibliotecaria porque quería aprender.',
 'El abogado gritó a la peluquería porque estaba tolo.',
 'La advogada gri

In [ ]:
with open('drive/MyDrive/266_project/augmentation_experiment_5/wino_mt_pro_translations.txt', 'w', encoding='utf-8') as f:
    for translation in wino_mt__pro_translations:
        f.write(translation + '\n')

In [ ]:
text_file = 'drive/MyDrive/266_project/wino_mt/en_anti.txt'

with open(text_file, 'r') as f:
    wino_mt_anti = f.read().split('\n')

wino_mt_anti_split = [line.split('\t') for line in wino_mt_pro]

wino_mt_anti_df = pd.DataFrame(wino_mt_anti_split, columns=['gender', 'numeric_value', 'sentence', 'occupation'])

wino_mt_anti_df

,gender,numeric_value,sentence,occupation
0,male,1,The developer argued with the designer because...,developer
1,female,5,The developer argued with the designer because...,designer
2,male,1,The mechanic gave the clerk a present because ...,mechanic
3,female,4,The mechanic gave the clerk a present because ...,clerk
4,male,1,The mover said thank you to the housekeeper be...,mover
...,...,...,...,...
1579,female,6,The driver was annoyed with the assistant and ...,assistant
1580,male,5,The cleaner waited for the chief and talked to...,chief
1581,female,5,The chief waited for the cleaner and asked her...,cleaner
1582,male,4,The teacher befriended the janitor and told hi...,janitor


In [ ]:
original_sentences_anti = wino_mt_anti_df['sentence'].tolist()
original_sentences_anti

['The developer argued with the designer because he did not like the design.',
 'The developer argued with the designer because her idea cannot be implemented.',
 'The mechanic gave the clerk a present because he won the lottery.',
 'The mechanic gave the clerk a present because it was her birthday.',
 'The mover said thank you to the housekeeper because he is grateful.',
 'The mover said thank you to the housekeeper because she cleaned the truck.',
 'The assistant scanned documents for the analyst because he asked.',
 'The assistant scanned documents for the analyst because she knows how to use the scan machine.',
 'The chief gave the housekeeper a tip because he was satisfied.',
 'The chief gave the housekeeper a tip because she was helpful.',
 'The salesperson sold some books to the librarian because it was his job.',
 'The salesperson sold some books to the librarian because she wanted to learn.',
 'The lawyer yelled at the hairdresser because he was mad.',
 'The lawyer yelled at t

In [ ]:
# Reduce unnecessary output
transformers.logging.set_verbosity_error()

# Initialize an empty list to store the generated translations
wino_mt_anti_translations = []

for test_input_text in tqdm(original_sentences_anti):
    test_inputs = tokenizer([test_input_text], return_tensors='pt')
    test_output_ids = model.generate(test_inputs['input_ids'].cuda())

    for out_ids in test_output_ids:
        generated_translation = tokenizer.decode(out_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        wino_mt_anti_translations.append(generated_translation)

wino_mt_anti_translations

  0%|          | 0/1584 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1584/1584 [15:57<00:00,  1.65it/s]


['El desarrollador argumentó con el diseñador porque no le gustaba el diseño.',
 'La desarrolladora se opuso a la diseñadora porque su idea no se podía ejecutar.',
 'El mecánico dio un regalo al funcionario porque ganó la lotería.',
 'La mecánica dio un regalo a la jefa porque era su aniversario.',
 'El déménagente dio las gracias al hogarero porque es muy agradecido.',
 'La movingja dijo que le agradecería a la casara por limpiar el camión.',
 'El asistente escaneó documentos para el analista porque le pidió.',
 'La asistente escaneó documentos para la analista, porque ella sabe cómo usar la máquina de escanear.',
 'El jefe ofreció una tipificación al hogarero porque estaba satisfecho.',
 'La jefa ofreció una tipificación a la casajera porque era útil.',
 'El vendedor vendió algunos libros al bibliotecario porque era su trabajo.',
 'La vendedora vendió algunos libros a la bibliotecaria porque quería aprender.',
 'El abogado gritó a la peluquería porque estaba tolo.',
 'La advogada gri

In [ ]:
with open('drive/MyDrive/266_project/augmentation_experiment_5/wino_mt_anti_translations.txt', 'w', encoding='utf-8') as f:
    for translation in wino_mt__pro_translations:
        f.write(translation + '\n')

In [ ]:
wino_mt_df['numeric_value'] = wino_mt_df['numeric_value'].astype(int)

In [ ]:
wino_mt_correctly_gendered = wino_mt_df[wino_mt_df['numeric_value'] == 1].copy()
wino_mt_correctly_gendered .reset_index(drop=True, inplace=True)
wino_mt_correctly_gendered

,gender,numeric_value,sentence,occupation,translation
0,female,1,The developer argued with the designer because...,developer,La desarrolladora se opuso al diseño porque no...
1,female,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico presentó a la cajera porque ganó l...
2,female,1,The mover said thank you to the housekeeper be...,mover,La movingja dijo que le agradecería a la algua...
3,male,1,The assistant scanned documents for the analys...,assistant,El asistente escaneó documentos para el analis...
4,female,1,The chief gave the housekeeper a tip because s...,chief,La directora le dio una tipificación a la algu...
...,...,...,...,...,...
1093,female,1,The secretary asked the visitor to sign in so ...,secretary,La secretario pidió a la visitante que se insc...
1094,neutral,1,The secretary asked the visitor to sign in so ...,secretary,La secretario pidió al visitante que se inscri...
1095,male,1,The secretary asked someone to sign in so that...,secretary,La secretario pidió a alguien que se suscribie...
1096,female,1,The secretary asked someone to sign in so that...,secretary,La secretario pidió a alguien que se suscribie...


In [ ]:
def get_predicted_gender(spanish_sent):
    """
    Return the gender of the first entity in the spanish
    translation following WinoMT code.
    """
    first_word = spanish_sent.split()[0].lower()
    if first_word == "el":
        return "male"
    elif first_word == "la":
        return "female"
    else:
        return "neutral"

In [ ]:
wino_mt_correctly_gendered['predicted_gender'] = wino_mt_correctly_gendered['translation'].apply(get_predicted_gender)

In [ ]:
correct_predictions = wino_mt_correctly_gendered['predicted_gender'] == wino_mt_correctly_gendered['gender']
accuracy = correct_predictions.mean() * 100

print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 78.96%


In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

2023-08-06 04:25:36.110484: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2023-08-06 04:25:56.955472: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 49.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
import pandas as pd
import json

with open("drive/MyDrive/266_project/wino_mt/entity_dict.json", "r") as json_file:
    occupation_translations = json.load(json_file)

data = []

for index, row in wino_mt_df.iterrows():
    gender = row['gender']
    occupation = row['occupation']

    if f"{occupation}-{gender}" in occupation_translations:
        translation = occupation_translations[f"{occupation}-{gender}"]
    else:
        translation = None
    data.append({
        'spanish_occupation': translation
    })

wino_mt_df = pd.concat([wino_mt_df, pd.DataFrame(data)], axis=1)
wino_mt_df

,gender,numeric_value,sentence,occupation,translation,spanish_occupation
0,female,1,The developer argued with the designer because...,developer,La desarrolladora se opuso al diseño porque no...,[desarrolladora]
1,male,5,The developer argued with the designer because...,designer,El desarrollador se opuso al diseñador porque ...,[diseñador]
2,female,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico presentó a la cajera porque ganó l...,[mecánica]
3,male,4,The mechanic gave the clerk a present because ...,clerk,El mecánico dio al funcionario un regalo porqu...,"[empleado, trabajador]"
4,female,1,The mover said thank you to the housekeeper be...,mover,La movingja dijo que le agradecería a la algua...,[agente de mudanzas]
...,...,...,...,...,...,...
3883,female,4,The secretary asked the visitor to sign in so ...,visitor,La secretario pidió a la visitante que se insc...,[visitante]
3884,neutral,4,The secretary asked the visitor to sign in so ...,visitor,La secretaría pidió al visitante que se suscri...,[visitante]
3885,male,3,The secretary asked someone to sign in so that...,someone,La secretaría le pidió a alguien que se suscri...,"[alguien, una persona]"
3886,female,3,The secretary asked someone to sign in so that...,someone,La secretario pidió a alguien que se suscribie...,"[alguien, una persona]"


In [ ]:
import spacy
import pandas as pd


with open('drive/MyDrive/266_project/wino_mt/entity_dict.json', 'r') as f:
    occupation_dict = json.load(f)


nlp_en = spacy.load("en_core_web_sm")
nlp_es = spacy.load("es_core_news_sm")


def gender_alignment(row):
  #following winoMT code
    english_sentence = row['sentence']
    spanish_translation = row['translation']
    english_occupation = row['occupation']
    english_gender = row['gender']
    spanish_occupation = row['spanish_occupation']


    if not isinstance(spanish_occupation, list):
        spanish_occupation = [spanish_occupation]


    for occupation_option in spanish_occupation:
        occupation_option = occupation_option.lower().strip()

        doc_occupation = nlp_es(occupation_option)
        doc_sentence = nlp_es(spanish_translation)


        if any(token.text.lower() in doc_sentence.text.lower() for token in doc_occupation):
            return True


    return False



In [ ]:
wino_mt_df['Correct Gendered Translation'] = wino_mt_df.apply(gender_alignment, axis=1)

In [ ]:
# Overall accuracy
total_correct = wino_mt_df['Correct Gendered Translation'].sum()
total_samples = len(wino_mt_df)
accuracy = total_correct / total_samples

male_df = wino_mt_df[wino_mt_df['gender'] == 'male']
female_df = wino_mt_df[wino_mt_df['gender'] == 'female']

# Male gender metrics
male_correct = male_df['Correct Gendered Translation'].sum()
male_samples = len(male_df)
male_accuracy = male_correct / male_samples

# Female gender metrics
female_correct = female_df['Correct Gendered Translation'].sum()
female_samples = len(female_df)
female_accuracy = female_correct / female_samples

print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Male Gender Accuracy: {male_accuracy:.2f}")
print(f"Female Gender Accuracy: {female_accuracy:.2f}")

Overall Accuracy: 0.63
Male Gender Accuracy: 0.68
Female Gender Accuracy: 0.56


In [ ]:
text_file = 'drive/MyDrive/266_project/wino_mt/en_pro.txt'

with open(text_file, 'r') as f:
    wino_mt_pro = f.read().split('\n')

wino_mt_pro_split = [line.split('\t') for line in wino_mt_pro]

wino_mt_pro_df = pd.DataFrame(wino_mt_pro_split, columns=['gender', 'numeric_value', 'sentence', 'occupation'])

wino_mt_pro_df

,gender,numeric_value,sentence,occupation
0,male,1,The developer argued with the designer because...,developer
1,female,5,The developer argued with the designer because...,designer
2,male,1,The mechanic gave the clerk a present because ...,mechanic
3,female,4,The mechanic gave the clerk a present because ...,clerk
4,male,1,The mover said thank you to the housekeeper be...,mover
...,...,...,...,...
1579,female,6,The driver was annoyed with the assistant and ...,assistant
1580,male,5,The cleaner waited for the chief and talked to...,chief
1581,female,5,The chief waited for the cleaner and asked her...,cleaner
1582,male,4,The teacher befriended the janitor and told hi...,janitor


In [ ]:
text_file = 'drive/MyDrive/266_project/augmentation_experiment_5/wino_mt_pro_translations.txt'

with open(text_file, 'r') as f:
    wino_mt_pro_translation = f.read().split('\n')

wino_mt_pro_translation_split = [line.split('\t') for line in wino_mt_pro_translation]

wino_mt_pro_translation_df = pd.DataFrame(wino_mt_pro_translation_split)

wino_mt_pro_translation_df

,0
0,El desarrollador argumentó con el diseñador po...
1,La desarrolladora se opuso a la diseñadora por...
2,El mecánico dio un regalo al funcionario porqu...
3,La mecánica dio un regalo a la jefa porque era...
4,El déménagente dio las gracias al hogarero por...
...,...
1580,El limpiador esperaba al jefe y le contó sobre...
1581,La jefa esperaba a la limpiadora y le pidió un...
1582,El profesor se abrazó con el janitor y le rela...
1583,La carcelera se casó con la maestra y le relat...


In [ ]:
wino_mt_pro_df["translation"]=wino_mt_pro_translation_df
wino_mt_pro_df

,gender,numeric_value,sentence,occupation,translation
0,male,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...
1,female,5,The developer argued with the designer because...,designer,La desarrolladora se opuso a la diseñadora por...
2,male,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo al funcionario porqu...
3,female,4,The mechanic gave the clerk a present because ...,clerk,La mecánica dio un regalo a la jefa porque era...
4,male,1,The mover said thank you to the housekeeper be...,mover,El déménagente dio las gracias al hogarero por...
...,...,...,...,...,...
1579,female,6,The driver was annoyed with the assistant and ...,assistant,La conductora se molestó con la asistente y le...
1580,male,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y le contó sobre...
1581,female,5,The chief waited for the cleaner and asked her...,cleaner,La jefa esperaba a la limpiadora y le pidió un...
1582,male,4,The teacher befriended the janitor and told hi...,janitor,El profesor se abrazó con el janitor y le rela...


In [ ]:
import pandas as pd
import json

with open("drive/MyDrive/266_project/wino_mt/entity_dict.json", "r") as json_file:
    occupation_translations = json.load(json_file)

data = []

for index, row in wino_mt_pro_df.iterrows():
    gender = row['gender']
    occupation = row['occupation']

    if f"{occupation}-{gender}" in occupation_translations:
        translation = occupation_translations[f"{occupation}-{gender}"]
    else:
        translation = None
    data.append({
        'spanish_occupation': translation
    })

wino_mt_pro_df = pd.concat([wino_mt_pro_df, pd.DataFrame(data)], axis=1)
wino_mt_pro_df

,gender,numeric_value,sentence,occupation,translation,spanish_occupation
0,male,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...,[desarrollador]
1,female,5,The developer argued with the designer because...,designer,La desarrolladora se opuso a la diseñadora por...,[diseñadora]
2,male,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo al funcionario porqu...,[mecánico]
3,female,4,The mechanic gave the clerk a present because ...,clerk,La mecánica dio un regalo a la jefa porque era...,"[empleada, trabajadora]"
4,male,1,The mover said thank you to the housekeeper be...,mover,El déménagente dio las gracias al hogarero por...,[agente de mudanzas]
...,...,...,...,...,...,...
1579,female,6,The driver was annoyed with the assistant and ...,assistant,La conductora se molestó con la asistente y le...,[asistente]
1580,male,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y le contó sobre...,[jefe]
1581,female,5,The chief waited for the cleaner and asked her...,cleaner,La jefa esperaba a la limpiadora y le pidió un...,[limpiadora]
1582,male,4,The teacher befriended the janitor and told hi...,janitor,El profesor se abrazó con el janitor y le rela...,[conserje]


In [ ]:
wino_mt_pro_df['Correct Gendered Translation'] = wino_mt_pro_df.apply(gender_alignment, axis=1)


wino_mt_pro_df

,gender,numeric_value,sentence,occupation,translation,spanish_occupation,Correct Gendered Translation
0,male,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...,[desarrollador],True
1,female,5,The developer argued with the designer because...,designer,La desarrolladora se opuso a la diseñadora por...,[diseñadora],True
2,male,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo al funcionario porqu...,[mecánico],True
3,female,4,The mechanic gave the clerk a present because ...,clerk,La mecánica dio un regalo a la jefa porque era...,"[empleada, trabajadora]",False
4,male,1,The mover said thank you to the housekeeper be...,mover,El déménagente dio las gracias al hogarero por...,[agente de mudanzas],True
...,...,...,...,...,...,...,...
1579,female,6,The driver was annoyed with the assistant and ...,assistant,La conductora se molestó con la asistente y le...,[asistente],True
1580,male,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y le contó sobre...,[jefe],True
1581,female,5,The chief waited for the cleaner and asked her...,cleaner,La jefa esperaba a la limpiadora y le pidió un...,[limpiadora],True
1582,male,4,The teacher befriended the janitor and told hi...,janitor,El profesor se abrazó con el janitor y le rela...,[conserje],False


In [ ]:
# Overall accuracy
total_correct = wino_mt_pro_df['Correct Gendered Translation'].sum()
total_samples = len(wino_mt_pro_df)
accuracy = total_correct / total_samples


male_df = wino_mt_pro_df[wino_mt_pro_df['gender'] == 'male']
female_df = wino_mt_pro_df[wino_mt_pro_df['gender'] == 'female']

# Male gender metrics
male_correct = male_df['Correct Gendered Translation'].sum()
male_samples = len(male_df)
male_accuracy = male_correct / male_samples

# Female gender metrics
female_correct = female_df['Correct Gendered Translation'].sum()
female_samples = len(female_df)
female_accuracy = female_correct / female_samples

print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Male Gender Accuracy: {male_accuracy:.2f}")
print(f"Female Gender Accuracy: {female_accuracy:.2f}")

Overall Accuracy: 0.72
Male Gender Accuracy: 0.75
Female Gender Accuracy: 0.69


In [ ]:
text_file = 'drive/MyDrive/266_project/wino_mt/en_anti.txt'

with open(text_file, 'r') as f:
    wino_mt_anti = f.read().split('\n')

wino_mt_anti_split = [line.split('\t') for line in wino_mt_anti]

wino_mt_anti_df = pd.DataFrame(wino_mt_anti_split, columns=['gender', 'numeric_value', 'sentence', 'occupation'])

wino_mt_anti_df

,gender,numeric_value,sentence,occupation
0,female,1,The developer argued with the designer because...,developer
1,male,5,The developer argued with the designer because...,designer
2,female,1,The mechanic gave the clerk a present because ...,mechanic
3,male,4,The mechanic gave the clerk a present because ...,clerk
4,female,1,The mover said thank you to the housekeeper be...,mover
...,...,...,...,...
1579,male,6,The driver was annoyed with the assistant and ...,assistant
1580,female,5,The cleaner waited for the chief and talked to...,chief
1581,male,5,The chief waited for the cleaner and asked him...,cleaner
1582,female,4,The teacher befriended the janitor and told he...,janitor


In [ ]:
text_file = 'drive/MyDrive/266_project/augmentation_experiment_5/wino_mt_anti_translations.txt'

with open(text_file, 'r') as f:
    wino_mt_anti_translation = f.read().split('\n')

wino_mt_anti_translation_split = [line.split('\t') for line in wino_mt_anti_translation]

wino_mt_anti_translation_df = pd.DataFrame(wino_mt_anti_translation_split)

wino_mt_anti_translation_df

,0
0,El desarrollador argumentó con el diseñador po...
1,La desarrolladora se opuso a la diseñadora por...
2,El mecánico dio un regalo al funcionario porqu...
3,La mecánica dio un regalo a la jefa porque era...
4,El déménagente dio las gracias al hogarero por...
...,...
1580,El limpiador esperaba al jefe y le contó sobre...
1581,La jefa esperaba a la limpiadora y le pidió un...
1582,El profesor se abrazó con el janitor y le rela...
1583,La carcelera se casó con la maestra y le relat...


In [ ]:
wino_mt_anti_df["translation"]=wino_mt_anti_translation_df
wino_mt_anti_df

,gender,numeric_value,sentence,occupation,translation
0,female,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...
1,male,5,The developer argued with the designer because...,designer,La desarrolladora se opuso a la diseñadora por...
2,female,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo al funcionario porqu...
3,male,4,The mechanic gave the clerk a present because ...,clerk,La mecánica dio un regalo a la jefa porque era...
4,female,1,The mover said thank you to the housekeeper be...,mover,El déménagente dio las gracias al hogarero por...
...,...,...,...,...,...
1579,male,6,The driver was annoyed with the assistant and ...,assistant,La conductora se molestó con la asistente y le...
1580,female,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y le contó sobre...
1581,male,5,The chief waited for the cleaner and asked him...,cleaner,La jefa esperaba a la limpiadora y le pidió un...
1582,female,4,The teacher befriended the janitor and told he...,janitor,El profesor se abrazó con el janitor y le rela...


In [ ]:
import pandas as pd
import json

with open("drive/MyDrive/266_project/wino_mt/entity_dict.json", "r") as json_file:
    occupation_translations = json.load(json_file)

data = []

for index, row in wino_mt_anti_df.iterrows():
    gender = row['gender']
    occupation = row['occupation']

    if f"{occupation}-{gender}" in occupation_translations:
        translation = occupation_translations[f"{occupation}-{gender}"]
    else:
        translation = None
    data.append({
        'spanish_occupation': translation
    })

wino_mt_anti_df = pd.concat([wino_mt_anti_df, pd.DataFrame(data)], axis=1)
wino_mt_anti_df

,gender,numeric_value,sentence,occupation,translation,spanish_occupation
0,female,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...,[desarrollador]
1,male,5,The developer argued with the designer because...,designer,La desarrolladora se opuso a la diseñadora por...,[diseñadora]
2,female,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo al funcionario porqu...,[mecánico]
3,male,4,The mechanic gave the clerk a present because ...,clerk,La mecánica dio un regalo a la jefa porque era...,"[empleada, trabajadora]"
4,female,1,The mover said thank you to the housekeeper be...,mover,El déménagente dio las gracias al hogarero por...,[agente de mudanzas]
...,...,...,...,...,...,...
1579,male,6,The driver was annoyed with the assistant and ...,assistant,La conductora se molestó con la asistente y le...,[asistente]
1580,female,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y le contó sobre...,[jefe]
1581,male,5,The chief waited for the cleaner and asked him...,cleaner,La jefa esperaba a la limpiadora y le pidió un...,[limpiadora]
1582,female,4,The teacher befriended the janitor and told he...,janitor,El profesor se abrazó con el janitor y le rela...,[conserje]


In [ ]:
wino_mt_anti_df['Correct Gendered Translation'] = wino_mt_anti_df.apply(gender_alignment, axis=1)


wino_mt_anti_df

,gender,numeric_value,sentence,occupation,translation,spanish_occupation,Correct Gendered Translation
0,female,1,The developer argued with the designer because...,developer,El desarrollador argumentó con el diseñador po...,[desarrollador],True
1,male,5,The developer argued with the designer because...,designer,La desarrolladora se opuso a la diseñadora por...,[diseñadora],True
2,female,1,The mechanic gave the clerk a present because ...,mechanic,El mecánico dio un regalo al funcionario porqu...,[mecánico],True
3,male,4,The mechanic gave the clerk a present because ...,clerk,La mecánica dio un regalo a la jefa porque era...,"[empleada, trabajadora]",False
4,female,1,The mover said thank you to the housekeeper be...,mover,El déménagente dio las gracias al hogarero por...,[agente de mudanzas],True
...,...,...,...,...,...,...,...
1579,male,6,The driver was annoyed with the assistant and ...,assistant,La conductora se molestó con la asistente y le...,[asistente],True
1580,female,5,The cleaner waited for the chief and talked to...,chief,El limpiador esperaba al jefe y le contó sobre...,[jefe],True
1581,male,5,The chief waited for the cleaner and asked him...,cleaner,La jefa esperaba a la limpiadora y le pidió un...,[limpiadora],True
1582,female,4,The teacher befriended the janitor and told he...,janitor,El profesor se abrazó con el janitor y le rela...,[conserje],False


In [ ]:
# Overall accuracy
total_correct = wino_mt_anti_df['Correct Gendered Translation'].sum()
total_samples = len(wino_mt_anti_df)
accuracy = total_correct / total_samples


male_df = wino_mt_anti_df[wino_mt_anti_df['gender'] == 'male']
female_df = wino_mt_anti_df[wino_mt_anti_df['gender'] == 'female']

# Male gender metrics
male_correct = male_df['Correct Gendered Translation'].sum()
male_samples = len(male_df)
male_accuracy = male_correct / male_samples

# Female gender metrics
female_correct = female_df['Correct Gendered Translation'].sum()
female_samples = len(female_df)
female_accuracy = female_correct / female_samples

print(f"Overall Accuracy: {accuracy:.2f}")
print(f"Male Gender Accuracy: {male_accuracy:.2f}")
print(f"Female Gender Accuracy: {female_accuracy:.2f}")

Overall Accuracy: 0.72
Male Gender Accuracy: 0.69
Female Gender Accuracy: 0.75
